In [25]:
import time
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler # 표준화(평균을 기준)
from sklearn.preprocessing import MinMaxScaler # 정규화(정해진 규칙 기준, mix-max기준)
from sklearn.datasets import load_iris
# cross_val_score: 다양한 샘플링으 ㄹ해서 여러번 테스트 --> 평균치를 구해본다.!
from sklearn.metrics import *
%matplotlib inline

# 주요 라이브러리 import

from matplotlib import rc, font_manager
import matplotlib.font_manager as fm


# 디렉토리 및 파일 이름에 맞추어 변경
font_location = '../NanumGothic.ttf'
font_name = fm.FontProperties(fname=font_location).get_name()
rc('font', family=font_name)
import scipy.io as spio



In [26]:
df = pd.read_excel('../csv-data/titanic.xls')
df

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",female,14.5000,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN
1305,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3,0,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN


In [27]:
## 1. p-class column update => 1:low, 2-3: middle, 4-5: high
df['pclass'] = df['pclass'].apply(lambda x: 'low' if x == 1 else ('middle' if x in [2,3] else ('high' if x in [4,5] else 0)))
df['pclass'].value_counts()


middle    986
low       323
Name: pclass, dtype: int64

In [28]:
## 2. name으로 파생변수 name_result를 만드세요
## 글자수를 세어 15이상이면long, 나머지 short
df['name_result'] = df['name'].apply(lambda x: 'long' if len(x)>=15 else 'short')
df['name_result'].value_counts()

long     1299
short      10
Name: name_result, dtype: int64

In [29]:
## 3. survived 1이면 생존, 0이면 사망, survived column update
df['survived'] = df['survived'].apply(lambda x: '생존' if x==1 else '사망')
df['survived'].value_counts()

사망    809
생존    500
Name: survived, dtype: int64

In [31]:
## 4. age 10대, 20대, 30대, 40대, ,50대, 60대 이상으로 column update
bins = [0, 10, 20, 30, 40, 50, 60, 150]
labels = ['아동', '10대', '20대', '30대', '40대', '50대', '60대 이상']
df['age'] = pd.cut(df.age, bins, labels = labels, include_lowest = True)
df['age'].value_counts()

20대       361
30대       210
10대       162
40대       132
아동         86
50대        62
60대 이상     33
Name: age, dtype: int64

In [32]:
## 5. name에 Mr, Dr 를 포함하고 있으면 '남자', Miss를 포함하고 있으면 '여자', 나머지는 '기타'
## 파생변수 name_result2 를 만들고, 각각을 카운트! 남자몇명, 여자 몇명??
df['name_result2'] = df['name'].apply(lambda x : '남자' if 'Mr' in x or 'Dr' in x else ('여자' if 'Miss' in x else '기타'))
df['name_result2'].value_counts()

남자    970
여자    256
기타     83
Name: name_result2, dtype: int64

In [33]:
df['name'].head(50)

0                         Allen, Miss. Elisabeth Walton
1                        Allison, Master. Hudson Trevor
2                          Allison, Miss. Helen Loraine
3                  Allison, Mr. Hudson Joshua Creighton
4       Allison, Mrs. Hudson J C (Bessie Waldo Daniels)
5                                   Anderson, Mr. Harry
6                     Andrews, Miss. Kornelia Theodosia
7                                Andrews, Mr. Thomas Jr
8         Appleton, Mrs. Edward Dale (Charlotte Lamson)
9                               Artagaveytia, Mr. Ramon
10                               Astor, Col. John Jacob
11    Astor, Mrs. John Jacob (Madeleine Talmadge Force)
12                        Aubart, Mme. Leontine Pauline
13                         Barber, Miss. Ellen "Nellie"
14                 Barkworth, Mr. Algernon Henry Wilson
15                                  Baumann, Mr. John D
16                             Baxter, Mr. Quigg Edmond
17      Baxter, Mrs. James (Helene DeLaudeniere 